## 데이터타입 바꾸는 코드

In [ ]:
import librosa
import soundfile as sf

def fix_wav(input_path, output_path):
    wav, sr = librosa.load(input_path, sr=16000, mono=True)
    sf.write(output_path, wav, 16000)

fix_wav("data/침착맨1.wav", "data/침착맨1_fixed.wav")
fix_wav("data/침착+나폴리mix.wav", "data/mix_fixed.wav")


/var/folders/zx/88x5v8pj1h3g803rndft70xr0000gn/T/ipykernel_2810/2177184664.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(input_path, sr=16000, mono=True)


## 두 음성파일 합치는 코드

In [2]:
import soundfile as sf
import numpy as np
import librosa

def load_wav(path, sr=16000):
    audio, orig_sr = librosa.load(path, sr=sr)  # 자동 리샘플링
    return audio

def mix_wavs(wav1_path, wav2_path, output_path="mixed.wav", sr=16000):
    # Load audio files
    wav1 = load_wav(wav1_path, sr)
    wav2 = load_wav(wav2_path, sr)

    # Match length by taking the shorter one
    min_len = min(len(wav1), len(wav2))
    wav1 = wav1[:min_len]
    wav2 = wav2[:min_len]

    # Mix (simple average or sum — choose one)
    
    # Option 1: Sum (louder)
    mixed = wav1 + wav2

    # Option 2: Average (prevent clipping)
    # mixed = (wav1 + wav2) / 2.0

    # Normalize to prevent clipping
    max_val = np.max(np.abs(mixed))
    if max_val > 1.0:
        mixed = mixed / max_val

    # Save as WAV
    sf.write(output_path, mixed, sr)
    print(f"Saved mixed audio to: {output_path}")


mix_wavs("test_data/진영웅 김지원.wav", "test_data/한로로 소스1.wav", "test_data/진영웅김지원한로로.wav")

/var/folders/zx/88x5v8pj1h3g803rndft70xr0000gn/T/ipykernel_24094/1346338055.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, orig_sr = librosa.load(path, sr=sr)  # 자동 리샘플링
/opt/anaconda3/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Saved mixed audio to: test_data/진영웅김지원한로로.wav


In [ ]:
import librosa
import soundfile as sf
import os

def split_audio_30s(input_path, output_dir, sr=16000, chunk_duration=30):
    # Load audio
    audio, sr = librosa.load(input_path, sr=sr)
    os.makedirs(output_dir, exist_ok=True)
    
    # 30초 → 샘플 개수로 변환
    chunk_samples = sr * chunk_duration
    total_samples = len(audio)
    
    # Split
    idx = 0
    for start in range(0, total_samples, chunk_samples):
        end = min(start + chunk_samples, total_samples)
        chunk = audio[start:end]
        
        # Save chunk
        out_path = os.path.join(output_dir, f"chunk_{idx}.wav")
        sf.write(out_path, chunk, sr)
        print(f"Saved: {out_path}")
        
        idx += 1
        break

# 사용 예시
split_audio_30s("test_data/카페 소음 1시간.wav", "test_data/카페소음 30초")


In [2]:
import soundfile as sf
import numpy as np
import librosa

def load_wav(path, sr=16000):
    audio, orig_sr = librosa.load(path, sr=sr)
    return audio

def rms(x):
    return np.sqrt(np.mean(x**2))

def normalize_to_rms(audio, target_rms=0.1):
    cur_rms = rms(audio)
    if cur_rms < 1e-9:
        return audio
    return audio * (target_rms / cur_rms)

def mix_3_wavs_equal_db(wav1_path, wav2_path, wav3_path, output_path="mixed.wav", sr=16000):
    # Load audio
    wav1 = load_wav(wav1_path, sr)
    wav2 = load_wav(wav2_path, sr)
    wav3 = load_wav(wav3_path, sr)

    # Match length
    min_len = min(len(wav1), len(wav2), len(wav3))
    wav1, wav2, wav3 = wav1[:min_len], wav2[:min_len], wav3[:min_len]

    # Normalize loudness (make dB/RMS equal)
    target_rms = 0.1    # 적당히 안정적인 음량 기준값
    wav1 = normalize_to_rms(wav1, target_rms)
    wav2 = normalize_to_rms(wav2, target_rms)
    wav3 = normalize_to_rms(wav3, target_rms)

    # Mix (sum)
    mixed = wav1 + wav2 + wav3

    # Normalize for clipping safeguard
    max_val = np.max(np.abs(mixed))
    if max_val > 1.0:
        mixed = mixed / max_val

    sf.write(output_path, mixed, sr)
    print(f"[✓] Saved mixed audio to: {output_path}")


# 실행 예시
mix_3_wavs_equal_db(
    "test_data/진영웅3.wav",
    "test_data/김지원3.wav",
    "test_data/이재현3.wav",
    "test_data/mix3_equal_db.wav"
)


[✓] Saved mixed audio to: test_data/mix3_equal_db.wav


/var/folders/zx/88x5v8pj1h3g803rndft70xr0000gn/T/ipykernel_81917/2349795843.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, orig_sr = librosa.load(path, sr=sr)
